In [2]:
import pymysql
import pymssql
import happybase
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [3]:
from IPython.display import display

# 조회한 데이터를 생략없이 전체 확인하기 위한 디스플레이 옵션 설정 
pd.options.display.max_rows = None
pd.options.display.max_columns = None

## pymssql / pymysql을 이용한 mssql / mysql DB 사용

#### 1. DB 접속

In [15]:
# conn 객체를 이용한 접속

# mssql
conn = pymssql.connect(database ='db_name', 
                       user='user_name', 
                       password='password', 
                       host='0.0.0.0', 
                       port='11223')

# mysql(charset 설정 utf8mb4)
conn = pymysql.connect(host='0.0.0.0', 
                       port=33306, 
                       user= 'user_name', 
                       password='password', 
                       db='db_name',
                       charset='utf8mb4')

#### 2. 쿼리 작성 : 테이블 조회

In [14]:
# 수행할 쿼리문 작성(fstring 사용)

# test_table으로부터 원하는 기간의 데이터를 조회할 경우
begin = '2020-06-01'
end = '2020-06-30'
table_name = 'test_table'; it_code=3; kind_code='001001001'

# 쿼리 작성 시작
qry = f"select * from {table_name} where datetime between '{begin}' and '{end}'"

# 조건절 추가 
cond1 = f" and where item_code={it_code}"
qry += cond1

cond2 = f" and where kind_code={kind_code}"
qry += cond2

# 정렬 :  최근 날짜 순으로 정렬(내림차순)
orderby = " order by datetime desc"
qry += orderby

print(qry)

select * from test_table where datetime between '2020-06-01' and '2020-06-30' and where item_code=3 and where kind_code=001001001 order by datetime desc


#### 3. 쿼리 수행 : pandas를 이용하여 조회된 데이터를 DataFrame 형식으로 저장

In [3]:
df = pd.read_sql(sql=qry, con=conn)
conn.close()

display(df)

,Kind_Code,Eng_ItCode,Eng_RDate,Daily_Usage,0000,0015,0030,0045,0100,0115,0130,0145,0200,0215,0230,0245,0300,0315,0330,0345,0400,0415,0430,0445,0500,0515,0530,0545,0600,0615,0630,0645,0700,0715,0730,0745,0800,0815,0830,0845,0900,0915,0930,0945,1000,1015,1030,1045,1100,1115,1130,1145,1200,1215,1230,1245,1300,1315,1330,1345,1400,1415,1430,1445,1500,1515,1530,1545,1600,1615,1630,1645,1700,1715,1730,1745,1800,1815,1830,1845,1900,1915,1930,1945,2000,2015,2030,2045,2100,2115,2130,2145,2200,2215,2230,2245,2300,2315,2330,2345
0,001001001,3,2020-06-01,None,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,28.179,115.604,142.582,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,142.575,141.695,114.104,98.369,95.557,95.185,99.774,106.092,117.258,92.211,63.710,76.090,83.132,78.550,65.300,65.049,67.058,55.544,54.123,47.986,51.157,55.657,45.092,38.662,43.490,41.806,33.765,28.259,47.809,28.986,28.706,32.239,38.896,28.259
1,001001001,3,2020-06-02,None,26.068,26.079,37.602,25.900,27.017,28.989,41.429,27.624,27.632,27.236,27.391,38.709,27.742,27.193,26.835,34.076,32.042,26.426,26.804,33.071,32.008,26.679,29.524,44.701,44.735,63.261,63.778,55.869,66.529,56.930,63.527,90.181,110.850,78.878,92.295,125.858,89.690,85.711,101.728,106.201,92.447,116.898,124.597,149.633,124.803,98.143,123.444,113.601,97.994,93.945,103.150,92.916,92.490,103.655,122.745,136.609,123.607,115.716,102.674,99.091,111.754,96.390,85.512,95.665,97.400,91.324,103.415,117.218,141.841,135.869,127.990,94.199,77.651,52.249,60.240,89.200,73.732,66.118,57.199,67.130,50.416,49.951,56.092,51.075,40.520,45.430,45.635,45.831,37.830,35.787,34.794,36.288,30.667,37.452,29.405,29.277
2,001001001,3,2020-06-03,None,43.107,30.014,27.714,30.266,27.523,36.695,30.373,28.173,27.462,30.140,37.966,32.661,29.996,28.192,38.173,28.281,27.736,27.588,37.915,34.645,27.931,30.851,28.948,36.574,55.458,57.288,72.096,57.010,58.462,55.462,63.907,74.201,111.879,90.936,86.773,90.943,98.764,108.383,104.563,130.351,136.010,131.650,139.937,151.901,139.988,136.506,132.518,148.851,158.989,175.748,150.328,153.693,156.999,159.293,179.757,195.922,197.631,174.875,169.843,182.197,157.888,158.768,143.119,145.482,125.657,133.994,126.510,88.481,78.590,66.777,76.991,71.300,53.287,50.528,77.258,69.026,50.252,43.672,47.381,51.197,41.172,41.308,37.818,37.847,42.914,37.275,34.854,31.465,34.780,39.868,30.558,29.984,36.429,30.263,27.485,27.409
3,001001001,3,2020-06-04,None,27.964,35.355,25.584,26.767,27.206,35.959,25.426,25.693,25.348,34.658,30.131,24.752,26.769,25.060,37.402,25.479,25.069,26.590,39.613,28.538,24.718,26.217,26.447,30.815,51.020,52.032,52.501,58.776,56.667,61.398,55.438,64.787,86.725,90.418,80.517,87.461,104.129,92.322,96.310,111.255,136.775,153.104,155.713,171.190,181.166,184.438,158.291,159.088,160.812,174.179,163.752,145.779,150.466,171.302,183.101,196.165,189.282,175.628,183.070,178.686,165.651,168.194,167.094,182.557,162.199,159.415,123.087,99.711,101.362,108.168,99.772,91.788,69.484,54.117,88.172,66.399,77.855,104.626,119.249,110.619,98.173,106.099,82.603,87.560,100.779,85.763,80.451,56.477,38.770,36.363,28.164,34.128,30.823,33.562,25.791,26.611
4,001001001,3,2020-06-05,None,25.925,34.374,24.594,24.593,32.483,34.485,24.187,24.798,27.183,24.500,37.810,24.445,25.454,25.670,28.992,32.685,24.658,24.667,28.083,37.034,25.687,25.421,26.682,31.817,49.441,55.950,51.151,46.019,42.427,48.651,48.705,58.555,89.731,97.525,87.056,97.798,138.078,152.843,158.582,160.148,160.660,175.411,195.356,207.892,183.537,175.517,179.468,163.461,163.468,180.945,170.143,167.134,169.857,203.820,202.393,194.622,174.568,181.018,171.977,180.553,161.581,162.060,163.824,140.771,124.428,119.709,114.597,96.701,106.956,106

#### 4. 기타 쿼리 (테이블 생성/삭제, 데이터 삽입/변경 등) : cursor 객체 사용

In [16]:
cursor = conn.cursor()

#### - 여러개의 데이터 한번에 삽입 : executemany()

In [48]:
# 삽입할 테이블의 컬럼에 맞게 튜플형태로 데이터 정리 
pet = pd.read_csv('Documents/pet_name.csv', encoding='utf-8-sig') # 한글(한자) 포함될 경우 encoding 주의
display(pet.head(3))

pet = pet.where(pd.notnull(pet), None) # insert sql문에서 사용할 수 있도록 null을 None으로 표현
print('* Null 표현 변경 : NaN -> None')
display(pet.head(3))

vals = [tuple(pet.iloc[i]) for i in range(len(pet))] # 테이블의 각 row(각 컬럼값을 tuple로 구성)에 삽입할 데이터 리스트 
print(f"* executemany로 삽입되는 데이터 :\n{vals[:3]}")

# 데이터 삽입
curosr.executemany(
f"INSERT INTO {table_name} VALUES ({vals})")

conn.commit()

,ani_id,ani_type,ani_color,gender,birthday,name
0,274,NaN,NaN,NaN,NaN,가오
1,275,NaN,NaN,NaN,NaN,가비
2,276,NaN,NaN,NaN,NaN,감


* Null 표현 변경 : NaN -> None


,ani_id,ani_type,ani_color,gender,birthday,name
0,274,None,None,None,None,가오
1,275,None,None,None,None,가비
2,276,None,None,None,None,감


* executemany로 삽입되는 데이터 :
[(274, None, None, None, None, '가오'), (275, None, None, None, None, '가비'), (276, None, None, None, None, '감')]


#### - create/update/alter 등 : sql을 cursor를 통해 실행 (+commit)

In [49]:
create_qry = f"""CREATE TABLE {table_name} (
                    {column1} VARCHAR(5) NOT NULL,
                    {column2} INT,
                    {column3} FLOAT,
                    {column4} VARCHAR(2),
                    {column5} VARCHAR(20)
                    PRIMARY KEY ({column1})
                    )"""

drop_qry = f"DROP TABLE {table_name}"

update_qry = f"UPDATE {table_name} SET {column_to_update}={new_value} WHERE {column}='{string_value}''"

alter_qry = f"ALTER TABLE {table_name} MODIFY {column} INT(5) NULL"

cursor.execute(create_qry)
cursor.execute(drop_qry)
cursor.execute(alter_qry)
cursor.execute(update_qry)

conn.commit()

## happybase로 HBase 접근
#### 수십만 ~수천만개 컬럼 사용 가능

#### 1. 접속 (conncection 객체 생성)

In [50]:
conn = happybase.Connection(host='0.0.0.0',autoconnect=True) # conn.open()
table = conn.table(f"{table_name}")

#### 2. 테이블 내용 조회 : scan table

In [105]:
# scan 함수 실행시 byte 타입으로 key와 value(type:dict)가 리턴된다. 
# scan(limit=1, reverse=True, row_start=rowkey 등등...)
temp = []
for key, data in table.scan():
    print(key, data)
    result = {"datetime":key.decode('utf-8','ignore'), "elec_usage":float(data[b'cf:elec']), "gas_usage":float(data[b'cf:gas'])}
    temp.append(result)

df = json_normalize(temp)

print(df)
conn.close()

b'2017-01-01 00:00' {b'cf:building': b'1', b'cf:elec': b'45.52', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 01:00' {b'cf:building': b'1', b'cf:elec': b'51.67', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 02:00' {b'cf:building': b'1', b'cf:elec': b'57.76', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 03:00' {b'cf:building': b'1', b'cf:elec': b'60.85', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 04:00' {b'cf:building': b'1', b'cf:elec': b'54.51', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 05:00' {b'cf:building': b'1', b'cf:elec': b'53.79', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 06:00' {b'cf:building': b'1', b'cf:elec': b'57.71', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 07:00' {b'cf:building': b'1', b'cf:elec': b'51.13', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 08:00' {b'cf:building': b'1', b'cf:elec': b'67.67', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-01-01 09:00' {b'cf:building': b'1', b'cf:elec': 

b'2017-02-11 16:00' {b'cf:building': b'1', b'cf:elec': b'10.04', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-11 17:00' {b'cf:building': b'1', b'cf:elec': b'7.24', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-11 18:00' {b'cf:building': b'1', b'cf:elec': b'18.72', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-11 19:00' {b'cf:building': b'1', b'cf:elec': b'56.09', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-11 20:00' {b'cf:building': b'1', b'cf:elec': b'50.99', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-11 21:00' {b'cf:building': b'1', b'cf:elec': b'66.95', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-11 22:00' {b'cf:building': b'1', b'cf:elec': b'54.33', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-11 23:00' {b'cf:building': b'1', b'cf:elec': b'65.86', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-12 00:00' {b'cf:building': b'1', b'cf:elec': b'97.63', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-02-12 01:00' {b'cf:building': b'1', b'cf:elec': b

b'2017-03-25 08:00' {b'cf:building': b'1', b'cf:elec': b'2.22', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 09:00' {b'cf:building': b'1', b'cf:elec': b'4.9', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 10:00' {b'cf:building': b'1', b'cf:elec': b'6.64', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 11:00' {b'cf:building': b'1', b'cf:elec': b'6.76', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 12:00' {b'cf:building': b'1', b'cf:elec': b'6.21', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 13:00' {b'cf:building': b'1', b'cf:elec': b'8.48', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 14:00' {b'cf:building': b'1', b'cf:elec': b'8.08', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 15:00' {b'cf:building': b'1', b'cf:elec': b'10.69', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 16:00' {b'cf:building': b'1', b'cf:elec': b'11.76', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-03-25 17:00' {b'cf:building': b'1', b'cf:elec': b'14.81'

b'2017-05-03 10:00' {b'cf:building': b'1', b'cf:elec': b'8.95', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 11:00' {b'cf:building': b'1', b'cf:elec': b'8.2', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 12:00' {b'cf:building': b'1', b'cf:elec': b'6.64', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 13:00' {b'cf:building': b'1', b'cf:elec': b'9.0', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 14:00' {b'cf:building': b'1', b'cf:elec': b'7.36', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 15:00' {b'cf:building': b'1', b'cf:elec': b'8.51', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 16:00' {b'cf:building': b'1', b'cf:elec': b'7.47', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 17:00' {b'cf:building': b'1', b'cf:elec': b'11.09', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 18:00' {b'cf:building': b'1', b'cf:elec': b'9.88', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-05-03 19:00' {b'cf:building': b'1', b'cf:elec': b'8.88', b

b'2017-06-11 06:00' {b'cf:building': b'1', b'cf:elec': b'2.94', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 07:00' {b'cf:building': b'1', b'cf:elec': b'3.72', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 08:00' {b'cf:building': b'1', b'cf:elec': b'4.52', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 09:00' {b'cf:building': b'1', b'cf:elec': b'5.48', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 10:00' {b'cf:building': b'1', b'cf:elec': b'6.32', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 11:00' {b'cf:building': b'1', b'cf:elec': b'4.98', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 12:00' {b'cf:building': b'1', b'cf:elec': b'7.86', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 13:00' {b'cf:building': b'1', b'cf:elec': b'9.68', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 14:00' {b'cf:building': b'1', b'cf:elec': b'14.56', b'cf:gas': b'0.0', b'cf:predict': b'2'}
b'2017-06-11 15:00' {b'cf:building': b'1', b'cf:elec': b'14.49'

...

b'2019-08-22 03:00' {b'cf:building': b'1', b'cf:elec': b'8.34', b'cf:gas': b'54.5', b'cf:predict': b'2'}
b'2019-08-22 04:00' {b'cf:building': b'1', b'cf:elec': b'6.69', b'cf:gas': b'55.25', b'cf:predict': b'2'}
b'2019-08-22 05:00' {b'cf:building': b'1', b'cf:elec': b'7.93', b'cf:gas': b'48.4', b'cf:predict': b'2'}
b'2019-08-22 06:00' {b'cf:building': b'1', b'cf:elec': b'18.55', b'cf:gas': b'58.51', b'cf:predict': b'2'}
b'2019-08-22 07:00' {b'cf:building': b'1', b'cf:elec': b'32.46', b'cf:gas': b'83.13', b'cf:predict': b'2'}
b'2019-08-22 08:00' {b'cf:building': b'1', b'cf:elec': b'76.04', b'cf:gas': b'129.4', b'cf:predict': b'2'}
b'2019-08-22 09:00' {b'cf:building': b'1', b'cf:elec': b'99.1', b'cf:gas': b'147.9', b'cf:predict': b'2'}
b'2019-08-22 10:00' {b'cf:building': b'1', b'cf:elec': b'119.98', b'cf:gas': b'214.27', b'cf:predict': b'2'}
b'2019-08-22 11:00' {b'cf:building': b'1', b'cf:elec': b'155.24', b'cf:gas': b'204.58', b'cf:predict': b'2'}
b'2019-08-22 12:00' {b'cf:building': b'

b'2019-10-02 19:00' {b'cf:building': b'1', b'cf:elec': b'53.31', b'cf:gas': b'3.26', b'cf:predict': b'2'}
b'2019-10-02 20:00' {b'cf:building': b'1', b'cf:elec': b'52.61', b'cf:gas': b'4.03', b'cf:predict': b'2'}
b'2019-10-02 21:00' {b'cf:building': b'1', b'cf:elec': b'53.4', b'cf:gas': b'3.33', b'cf:predict': b'2'}
b'2019-10-02 22:00' {b'cf:building': b'1', b'cf:elec': b'33.83', b'cf:gas': b'3.15', b'cf:predict': b'2'}
b'2019-10-02 23:00' {b'cf:building': b'1', b'cf:elec': b'30.29', b'cf:gas': b'1.88', b'cf:predict': b'2'}
b'2019-10-03 00:00' {b'cf:building': b'1', b'cf:elec': b'102.32', b'cf:gas': b'4.74', b'cf:predict': b'2'}
b'2019-10-03 01:00' {b'cf:building': b'1', b'cf:elec': b'6.66', b'cf:gas': b'1.42', b'cf:predict': b'2'}
b'2019-10-03 02:00' {b'cf:building': b'1', b'cf:elec': b'5.39', b'cf:gas': b'1.08', b'cf:predict': b'2'}
b'2019-10-03 03:00' {b'cf:building': b'1', b'cf:elec': b'4.02', b'cf:gas': b'1.3', b'cf:predict': b'2'}
b'2019-10-03 04:00' {b'cf:building': b'1', b'cf:el

b'2019-11-08 16:00' {b'cf:building': b'1', b'cf:elec': b'54.71', b'cf:gas': b'6.26', b'cf:predict': b'2'}
b'2019-11-08 17:00' {b'cf:building': b'1', b'cf:elec': b'90.47', b'cf:gas': b'6.86', b'cf:predict': b'2'}
b'2019-11-08 18:00' {b'cf:building': b'1', b'cf:elec': b'47.89', b'cf:gas': b'4.11', b'cf:predict': b'2'}
b'2019-11-08 19:00' {b'cf:building': b'1', b'cf:elec': b'52.74', b'cf:gas': b'5.3', b'cf:predict': b'2'}
b'2019-11-08 20:00' {b'cf:building': b'1', b'cf:elec': b'52.51', b'cf:gas': b'4.38', b'cf:predict': b'2'}
b'2019-11-08 21:00' {b'cf:building': b'1', b'cf:elec': b'49.27', b'cf:gas': b'4.9', b'cf:predict': b'2'}
b'2019-11-08 22:00' {b'cf:building': b'1', b'cf:elec': b'41.68', b'cf:gas': b'3.5', b'cf:predict': b'2'}
b'2019-11-08 23:00' {b'cf:building': b'1', b'cf:elec': b'43.7', b'cf:gas': b'2.66', b'cf:predict': b'2'}
b'2019-11-09 00:00' {b'cf:building': b'1', b'cf:elec': b'107.96', b'cf:gas': b'5.75', b'cf:predict': b'2'}
b'2019-11-09 01:00' {b'cf:building': b'1', b'cf:e

b'2019-12-23 16:00' {b'cf:building': b'1', b'cf:elec': b'57.24', b'cf:gas': b'103.56', b'cf:predict': b'2'}
b'2019-12-23 17:00' {b'cf:building': b'1', b'cf:elec': b'53.81', b'cf:gas': b'90.85', b'cf:predict': b'2'}
b'2019-12-23 18:00' {b'cf:building': b'1', b'cf:elec': b'51.4', b'cf:gas': b'85.87', b'cf:predict': b'2'}
b'2019-12-23 19:00' {b'cf:building': b'1', b'cf:elec': b'57.22', b'cf:gas': b'85.95', b'cf:predict': b'2'}
b'2019-12-23 20:00' {b'cf:building': b'1', b'cf:elec': b'54.46', b'cf:gas': b'70.93', b'cf:predict': b'2'}
b'2019-12-23 21:00' {b'cf:building': b'1', b'cf:elec': b'55.34', b'cf:gas': b'51.48', b'cf:predict': b'2'}
b'2019-12-23 22:00' {b'cf:building': b'1', b'cf:elec': b'32.31', b'cf:gas': b'61.96', b'cf:predict': b'2'}
b'2019-12-23 23:00' {b'cf:building': b'1', b'cf:elec': b'27.07', b'cf:gas': b'55.4', b'cf:predict': b'2'}
b'2019-12-24 00:00' {b'cf:building': b'1', b'cf:elec': b'5.69', b'cf:gas': b'148.46', b'cf:predict': b'2'}
b'2019-12-24 01:00' {b'cf:building': b

               datetime  elec_usage  gas_usage
0   2017-01-01 00:00:00       45.52        0.0
1   2017-01-01 01:00:00       51.67        0.0
2   2017-01-01 02:00:00       57.76        0.0
3   2017-01-01 03:00:00       60.85        0.0
4   2017-01-01 04:00:00       54.51        0.0
5   2017-01-01 05:00:00       53.79        0.0
6   2017-01-01 06:00:00       57.71        0.0
7   2017-01-01 07:00:00       51.13        0.0
8   2017-01-01 08:00:00       67.67        0.0
9   2017-01-01 09:00:00       59.46        0.0
10  2017-01-01 10:00:00       43.91        0.0
11  2017-01-01 11:00:00        4.36        0.0
12  2017-01-01 12:00:00        4.52        0.0
13  2017-01-01 13:00:00        6.59        0.0
14  2017-01-01 14:00:00        7.01        0.0
15  2017-01-01 15:00:00        9.51        0.0
16  2017-01-01 16:00:00        9.80        0.0
17  2017-01-01 17:00:00        8.70        0.0
18  2017-01-01 18:00:00        4.13        0.0
19  2017-01-01 19:00:00        4.14        0.0
20  2017-01-0

In [77]:
from dateutil.relativedelta import relativedelta

# 최근 3개월 동안의 데이터 조회 (yymmdd 형식의 리스트 생성) <- timedelta는 month를 더하거나 뺄 수 없으므로 relativedelta 사용
now = datetime.now()
begin = now - relativedelta(months = 3)
num_of_day = (now-begin).days
print(num_of_day)

# strftime 함수를 이용하여 datetime 객체를 string으로 변형(yymmdd)
day_list = [datetime.strftime(begin+timedelta(days=i), '%y%m%d') for i in range(num_of_day) ]
print(day_list)


89
['210225', '210226', '210227', '210228', '210301', '210302', '210303', '210304', '210305', '210306', '210307', '210308', '210309', '210310', '210311', '210312', '210313', '210314', '210315', '210316', '210317', '210318', '210319', '210320', '210321', '210322', '210323', '210324', '210325', '210326', '210327', '210328', '210329', '210330', '210331', '210401', '210402', '210403', '210404', '210405', '210406', '210407', '210408', '210409', '210410', '210411', '210412', '210413', '210414', '210415', '210416', '210417', '210418', '210419', '210420', '210421', '210422', '210423', '210424', '210425', '210426', '210427', '210428', '210429', '210430', '210501', '210502', '210503', '210504', '210505', '210506', '210507', '210508', '210509', '210510', '210511', '210512', '210513', '210514', '210515', '210516', '210517', '210518', '210519', '210520', '210521', '210522', '210523', '210524']
